# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686582


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:07<01:39,  3.55s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:08,  2.54s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:48,  1.87s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:34,  1.39s/it]

...

Rendering models:  80%|████████  | 24/30 [00:11<00:01,  5.14it/s]

Rendering models:  83%|████████▎ | 25/30 [00:11<00:00,  5.23it/s]

Rendering models:  87%|████████▋ | 26/30 [00:11<00:00,  5.25it/s]

Rendering models:  93%|█████████▎| 28/30 [00:11<00:00,  6.04it/s]

Anfrily                               0.000222
not-logged-in-20a275b53abec8db9f41    0.065120
shirogane                             0.000225
richiemac                             0.000353
not-logged-in-a9247261537fa1da33fe    0.419338
Dynamatt                              0.000285
CThomas                               0.000945
katja72381                            0.000183
austingasbarra                        0.000580
pollux2728                            0.007186
Jack_07                               0.051478
not-logged-in-5958ad649776241f8a05    0.000228
nora.zoo                              0.000360
Evan.wickham                          0.010332
OlEnglish                             0.001163
not-logged-in-dddbb97cfd84354416f5    0.000222
homant                                0.000176
databanana                            0.000261
clairedeu                             0.000795
acapirala                             0.025034
rhondacard                            0.002236
Lavadude     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())